In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
df_directorio = pd.read_excel("DIRECTORIO DE OFICINAS.xlsx")
df_base = pd.read_excel("BASE CORREOS.xlsx")

In [3]:
df_directorio.head(3)

,Registro,Nombre,Correo,Puesto,División,ID Oficina,Nombre Oficina
0,P015072,"GANOZA HONORES, NATALIE ELIZABETH",NGANOZA@BBVA.COM,(B417)SUB GERENTE DE OFICINA F,DIVISION MIRAFLORES,564,OFICINA PLAZA BUTTERS
1,P016944,"QUISPE MAMANI, LUZ MERI",LMQUISPE@BBVA.COM,(B217)GERENTE DE OFICINA F,DIVISION SUR,763,OFICINA PEDREGAL
2,P016949,"TORRES PAREDES, GUISSEL PIERANGELY",GUISSELPIERANGELY.TORRES@BBVA.COM,(B252)EJECUTIVO BANCA NEGOCIOS 3,DIVISION SUR,763,OFICINA PEDREGAL


In [4]:
df_base.drop(df_base.columns[-1], axis=1, inplace=True)
df_base.head(3)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, eyttus...",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,5,"GCASTROP@BBVA.COM, buzon-procesal-sin-garantia...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com, paul...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA"


In [5]:
print('df_directorio: ', df_directorio.shape)
print('df_base: ', df_base.shape)

df_directorio:  (2944, 7)
df_base:  (333, 7)


In [6]:
df_directorio.isna().sum()

Registro          179
Nombre            179
Correo            181
Puesto              0
División            0
ID Oficina          0
Nombre Oficina    179
dtype: int64

In [7]:
df_base.isna().sum()

OFICINA           0
CORREOS           0
Territorio        1
Nombre Oficina    1
Cargo             1
Registro          1
Nombre            0
dtype: int64

In [8]:
df_directorio.dropna(inplace=True)
df_base.dropna(inplace=True)

print('df_directorio: ', df_directorio.shape)
print('df_base: ', df_base.shape)

df_directorio:  (2763, 7)
df_base:  (332, 7)


In [9]:
columnas_directorio = ['ID Oficina', 'Puesto', 'Correo']
df_directorio = df_directorio[columnas_directorio]
df_directorio.rename(columns={'ID Oficina': 'OFICINA'}, inplace=True)
df_directorio.sort_values(by='OFICINA', inplace=True, ignore_index=True)
df_directorio.reset_index(drop=True, inplace=True)
df_directorio.head(3)

,OFICINA,Puesto,Correo
0,2,(B321)ASESOR SERV JORNADA COMPLET 2,ALEXA.SIALER@BBVA.COM
1,2,(B252)EJECUTIVO BANCA NEGOCIOS 3,EYTTUSACA@BBVA.COM
2,2,(B217)GERENTE DE OFICINA F,TEYZAGUIRRE@BBVA.COM


In [10]:
df_directorio = df_directorio[df_directorio['Puesto'].str.contains('GERENTE', case=False)].reset_index(drop=True)
df_directorio.head(3)

,OFICINA,Puesto,Correo
0,2,(B217)GERENTE DE OFICINA F,TEYZAGUIRRE@BBVA.COM
1,2,(B417)SUB GERENTE DE OFICINA F,DMALAGA@BBVA.COM
2,5,(B417)SUB GERENTE DE OFICINA F,KGALVEZ@BBVA.COM


In [11]:
def concatenar_correos(grupo: pd.DataFrame) -> pd.Series:
    gerente_correo = grupo.loc[~grupo['Puesto'].str.contains('SUB GERENTE', case=False), 'Correo'].values
    subgerente_correo = grupo.loc[grupo['Puesto'].str.contains('SUB GERENTE', case=False), 'Correo'].values
    
    if len(gerente_correo) > 0 and len(subgerente_correo) > 0:
        correos_concatenados = gerente_correo[0] + ', ' + subgerente_correo[0]
    elif len(gerente_correo) > 0:
        correos_concatenados = gerente_correo[0]
    else:
        correos_concatenados = subgerente_correo[0]
    
    return pd.Series({
        'OFICINA': grupo['OFICINA'].iloc[0],
        'Puesto': 'GERENTE DE OFICINA',
        'Correo': correos_concatenados,
        })

df_final = df_directorio.groupby('OFICINA').apply(concatenar_correos).reset_index(drop=True)

print(df_final.shape)
df_final.head(3)

(289, 3)


,OFICINA,Puesto,Correo
0,2,GERENTE DE OFICINA,"TEYZAGUIRRE@BBVA.COM, DMALAGA@BBVA.COM"
1,5,GERENTE DE OFICINA,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM"
2,10,GERENTE DE OFICINA,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM"


In [12]:
df_actualizado = pd.merge(df_base, df_final, on='OFICINA', how='left')
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 9)


,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre,Puesto,Correo
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, eyttus...",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA",GERENTE DE OFICINA,"TEYZAGUIRRE@BBVA.COM, DMALAGA@BBVA.COM"
1,5,"GCASTROP@BBVA.COM, buzon-procesal-sin-garantia...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA",GERENTE DE OFICINA,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com, paul...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA",GERENTE DE OFICINA,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM"


In [13]:
df_actualizado.isna().sum()

OFICINA            0
CORREOS            0
Territorio         0
Nombre Oficina     0
Cargo              0
Registro           0
Nombre             0
Puesto            52
Correo            52
dtype: int64

In [14]:
df_actualizado['CORREOS'] = df_actualizado['Correo'].fillna(df_actualizado['CORREOS'])
df_actualizado.drop(columns={'Puesto', 'Correo'}, inplace=True)
df_actualizado['CORREOS'] = df_actualizado['CORREOS'].str.lower()
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 7)


,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,5,"jvegac@bbva.com, kgalvez@bbva.com",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA"


In [15]:
df_actualizado.isna().sum()

OFICINA           0
CORREOS           0
Territorio        0
Nombre Oficina    0
Cargo             0
Registro          0
Nombre            0
dtype: int64

In [16]:
df_actualizado.to_excel('CORREOS.xlsx', index=False)